# Scraper
### Notebook con el scraper para armar el dataset de noticias de Venezuela
## Features iniciales
### titulo: titulo de la noticia
### texto: cuerpo de la noticia
### fecha: fecha de publicación de a noticia
### link: dirección url de la noticia

In [1]:
import feedparser as fp
import json
import newspaper
import datetime 
import dateutil.parser
from datetime import date, timedelta
from newspaper import Article
from time import mktime
from datetime import datetime

In [2]:
def getDateTimeFromISO8601String(iso): 
    f = dateutil.parser.parse(iso) 
    return f

In [3]:
# Seteo del límite de la cantidad de noticias por sitio web
LIMIT = 100

data = {}
data['newspapers'] = {}

In [4]:
# Cargar el JSON de archivos con las urls a los diarios en línea
with open('NewsPapers.json') as data_file:
    companies = json.load(data_file)

In [5]:
count = 1

# Iteración por cada diario electrónico
for company, value in companies.items():
    print("Construyendo el sitio para ", company)
    paper = newspaper.build(value['link'], memoize_articles=False)
    newsPaper = {
        "link": value['link'],
        "articles": []
    }
    print("Sitio construido para ", company)

Construyendo el sitio para  cnn
Sitio construido para  cnn
Construyendo el sitio para  bbc
Sitio construido para  bbc
Construyendo el sitio para  universo
Sitio construido para  universo


In [23]:
    noneTypeCount = 0
    for content in paper.articles:
        if count > LIMIT:
            break
        try:
            content.download()
            content.parse()
        except Exception as e:
            print(e)
            print("continuing...")
            continue
        # Se saltará el artículo que no posea fecha de publicación.
        # Después de tener 10 artículos sin fecha, el diario electrónico será saltado.
        if content.publish_date is None:
            print(count, " Article has date of type None...")
            noneTypeCount = noneTypeCount + 1
            if noneTypeCount > 10:
                print("Too many noneType dates, aborting...")
                noneTypeCount = 0
                break
            count = count + 1
            continue
        else:
            fecha = getDateTimeFromISO8601String(content.publish_date.isoformat())
            fecha_desde = datetime(2018, 1, 1, 0, 0, 0)
            fecha_hasta = datetime(2018,12,31,23,59,59)
            palabras = ["Maduro","Venezuela"]
            if content.title in palabras:
                if fecha <= fecha_hasta and fecha >= fecha_desde:
                    article = {}
                    article['titulo'] = content.title
                    article['texto'] = content.text
                    article['link'] = content.url
                    article['fecha'] = content.publish_date.isoformat()
                    newsPaper['articles'].append(article)
                    print("Van ",count, " artículos bajados de ", company, " usando el sitio de url: ", content.url)
                    count = count + 1
                    noneTypeCount = 0

2019-02-16T00:00:00
1 articles downloaded from universo  using newspaper, url:  https://www.eluniverso.com/noticias/2019/02/16/nota/7192184/arabia-saudita-defiende-aplicacion-que-permite-hombres-vigilar
2019-02-16T00:00:00
2 articles downloaded from universo  using newspaper, url:  https://www.eluniverso.com/noticias/2019/02/16/nota/7191998/historia-como-se-desbarato-acuerdo-llegada-amazon-nueva-york
2019-02-16T00:00:00
3 articles downloaded from universo  using newspaper, url:  https://www.eluniverso.com/noticias/2019/02/16/nota/7191953/chalecos-amarillos-cumplen-3-meses-protestas-menos-manifestantes
2019-02-16T00:00:00
4 articles downloaded from universo  using newspaper, url:  https://www.eluniverso.com/noticias/2019/02/16/nota/7191911/nicaragua-oea-acuerdan-cooperar-reformas-proximas-elecciones
2019-02-16T00:00:00
5 articles downloaded from universo  using newspaper, url:  https://www.eluniverso.com/noticias/2019/02/16/nota/7191818/analizan-nuevas-directrices-sobre-bodas-menores
20

2019-02-14T00:00:00
43 articles downloaded from universo  using newspaper, url:  https://www.eluniverso.com/noticias/2019/02/14/nota/7188599/diputado-opositor-dice-que-se-incluira-militares-chavismo-maduro
2019-02-16T00:00:00
44 articles downloaded from universo  using newspaper, url:  https://www.eluniverso.com/opinion/2019/02/16/nota/7190696/oncologia-pediatrica
2019-02-15T00:00:00
45 articles downloaded from universo  using newspaper, url:  https://www.eluniverso.com/opinion/2019/02/15/nota/7189349/armas-carcel
2019-02-14T00:00:00
46 articles downloaded from universo  using newspaper, url:  https://www.eluniverso.com/opinion/2019/02/14/nota/7187543/visibilizarlos-sienta-precedente
2019-02-13T00:00:00
47 articles downloaded from universo  using newspaper, url:  https://www.eluniverso.com/opinion/2019/02/13/nota/7186232/cierre-pista
2019-02-12T00:00:00
48 articles downloaded from universo  using newspaper, url:  https://www.eluniverso.com/opinion/2019/02/12/nota/7185638/licitud-fondos

2018-12-30T00:00:00
90 articles downloaded from universo  using newspaper, url:  https://www.eluniverso.com/larevista/2018/12/30/nota/7114347/choquequirao-secreto
2018-12-30T00:00:00
91 articles downloaded from universo  using newspaper, url:  https://www.eluniverso.com/larevista/2018/12/30/nota/7114353/olon-siempre
2018-12-23T00:00:00
92 articles downloaded from universo  using newspaper, url:  https://www.eluniverso.com/larevista/2018/12/23/nota/7106751/tiempo-compartir
2019-02-20T00:00:00
93 articles downloaded from universo  using newspaper, url:  https://www.eluniverso.com/2019/02/20/evento/7187531/presentacion-samsung
2019-02-15T00:00:00
94 articles downloaded from universo  using newspaper, url:  https://www.eluniverso.com/larevista/2019/02/15/nota/7189211/agenda
2019-02-13T00:00:00
95 articles downloaded from universo  using newspaper, url:  https://www.eluniverso.com/larevista/2019/02/13/nota/7187144/fox-terrier-pelo-duro-es-coronado-show-westminster-nueva-york
2019-02-10T00:0

In [ ]:
data['newspapers'][company] = newsPaper
try:
    with open('scraped_articles.json', 'w') as outfile:
        json.dump(data, outfile)
except Exception as e: print(e)